In [1]:
import numpy as np
import pandas as pd
import sqlite3
import datetime

from utils import round_integers, today_date_as_epoch

grajki = [
    "ESTERAD#8174023",
    "NOTEVIDIUS#5532788",
    "BORUCHOMIR#7172961",
    "DAWNY-KECZUP#7192166",
    "GITAN#2046256",
    "WORRIED-BLADE115#1720700"
]

def aggregate_stats(data, player_name, last_hours=24):
    df = data[(data['utcStartSeconds'] > today_date_as_epoch()) & (data['player'] == player_name)]
    # df = data[(data['player'] == player_name)]
    
    result_meta = pd.DataFrame({
        "matches": df["matchID"].nunique(),
        "highestMultikill": df["highestMultikill"].max(),
        "scorePerMinuteAvg": df["scorePerMinute"].mean(),
        "highestStreak": df["highestStreak"].max(),
        "mostKillsInMatch": df['kills'].max(),
        "mostDeathsInMatch": df['deaths'].max(),
        "mostObjectivesInMatch": df['objectives'].max()
    }, index=[0])
    sums = pd.DataFrame(df[['kills', 'deaths', 'score', 'damageDealt', 'objectives', 'headshots', 'assists',
               'multikills', 'shotsFired', 'shotsLanded', 'shotsMissed', 'hits', 'timePlayed',
               'suicides', 'shots', 'timePlayedAlive', 'ekia']].sum()).T
    result = pd.concat([result_meta, sums], axis=1)
    result['kdRatio'] = round(result['kills'] / result['deaths'], 3)
    result['ekiadRatio'] = round(result['ekia'] / result['deaths'], 3)
    result['accuracy'] = round(result['shotsLanded'] / result['shotsFired'], 3)
    result['matchesWon'] = df[df["result"] == "win"].shape[0]
    result['scorePerMatchAvg'] = round(result['score'] / result['matches'], -1)
    result['killsPerMatchAvg'] = round(result['kills'] / result['matches'], 2)
    result = round_integers(result)
    return result[['matches', 'matchesWon', 'kills', 'deaths', 'kdRatio', 'ekiadRatio', 'assists',
                   'scorePerMatchAvg', 'killsPerMatchAvg',
                   'objectives', 'headshots',
                   'highestMultikill', 'highestStreak', 'scorePerMinuteAvg',
                   'score', 'damageDealt',
                   'shotsFired', 'shotsLanded', 'accuracy',
                   'suicides', 'mostKillsInMatch',
                   'mostDeathsInMatch', 'mostObjectivesInMatch']]

In [2]:
result = pd.DataFrame()
conn = sqlite3.connect("data/cod_stats.db")
df = pd.read_sql(
    sql=f"SELECT * FROM stats",
    con=conn
)

for player_id in grajki:
    player_name = player_id.split("#")[0].lower().replace("-", "_")
    df3 = aggregate_stats(df, player_name)
    df3.index = [player_name]
    result = pd.concat([result, df3])

for col in result.columns:
    result[col] = result[col].astype(str)
result = result.T

In [3]:
result

,esterad,notevidius,boruchomir,dawny_keczup,gitan,worried_blade115
matches,6,9,6,0,10,0
matchesWon,2,4,2,0,4,0
kills,154,117,48,0,277,0
deaths,127,160,99,0,199,0
kdRatio,1.213,0.731,0.485,0.0,1.392,0.0
ekiadRatio,1.457,0.831,0.545,0.0,1.563,0.0
assists,31,16,6,0,34,0
scorePerMatchAvg,4040,1920,890,0,4870,0
killsPerMatchAvg,25,13,8,0,27,0
objectives,61,29,14,0,57,0


In [5]:
from utils import round_integers

records_cols_dict = {
        "highestScore": "score",
        "highestDamageDealt": "damageDealt",
        "highestKdRatio": "kdRatio",
        "highestEkiadRatio": "ekiadRatio",
        "highestMultikill": "highestMultikill",
        "highestScorePerMinute": "scorePerMinute",
        "highestStreak": "highestStreak",
        "mostKillsInMatch": 'kills',
        "mostDeathsInMatch": 'deaths',
        "mostObjectivesInMatch": 'objectives',
        "mostHeadshotsInMatch": "headshots",
        "mostEkiaInMatch": "ekia"
}


def calc_records(data):
    df = round_integers(data).copy()
    records = pd.DataFrame({
        "highestScore": str(df["score"].max()),
        "highestDamageDealt": str(df["damageDealt"].max()),
        "highestKdRatio": str(df["kdRatio"].max()),
        "highestEkiadRatio": str(df["ekiadRatio"].max()),
        "highestMultikill": str(df["highestMultikill"].max()),
        "highestScorePerMinute": str(df["scorePerMinute"].max()),
        "highestStreak": str(df["highestStreak"].max()),
        "mostKillsInMatch": str(df['kills'].max()),
        "mostEkiaInMatch": str(df['ekia'].max()),
        "mostDeathsInMatch": str(df['deaths'].max()),
        "mostObjectivesInMatch": str(df['objectives'].max()),
        "mostHeadshotsInMatch": str(df["headshots"].max())
    }, index=["record"])
    records = records.T
    records['player'] = ''
    records['date'] = ''
    records['map'] = ''
    records['mode'] = ''
    for record in records.index:
        record_col = records_cols_dict[record]
        for desc_col in ['player', 'date', 'map', 'mode']:
            records[desc_col][record] = df[df[record_col].astype(str) == records.loc[record]['record']][desc_col].values[0]

    today_date = datetime.date.today().strftime("%Y-%m-%d")

    records['new'] = np.where(records['date'] == today_date, '<-- NOWY REKORD', '')
    
    return records

all_data = df.copy()
records_df = calc_records(all_data)

In [6]:
records_df

,record,player,date,map,mode,new
highestScore,14630,gitan,2023-10-15,mp_village_rm,dem,
highestDamageDealt,13250,esterad,2023-10-07,mp_cliffhanger,dem,
highestKdRatio,9.0,dawny_keczup,2023-10-03,mp_apocalypse,dem,
highestEkiadRatio,11.0,dawny_keczup,2023-09-24,mp_hijacked_rm,vip,
highestMultikill,7,esterad,2023-10-30,mp_slums_rm,dem,<-- NOWY REKORD
highestScorePerMinute,1359,esterad,2023-09-21,mp_tundra,conf,
highestStreak,15,gitan,2023-10-11,mp_mall,dem,
mostKillsInMatch,77,esterad,2023-10-07,mp_cliffhanger,dem,
mostEkiaInMatch,92,gitan,2023-10-26,mp_paintball_rm,dem,
mostDeathsInMatch,56,gitan,2023-10-01,mp_cliffhanger,dem,


---
## Individual records

In [7]:
def get_individual_records_query(record_col: str, record_name: str) -> str:
    return f"""
    WITH
    records AS (
        SELECT player, MAX({record_col}) AS {record_name}
        FROM stats
        GROUP BY 1
        ORDER BY 2 DESC
    )
    SELECT s.player, s.map, s.mode, s.date, r.{record_name}
    FROM stats s
    JOIN records r
        ON s.player = r.player
        AND s.{record_col} = r.{record_name}
    ORDER BY r.{record_name} DESC, s.date ASC;
    """

In [12]:
conn = sqlite3.connect("data/cod_stats.db")

for record_name in records_cols_dict:
    if record_name == 'mostDeathsInMatch':
        record_col = records_cols_dict[record_name]
    
        df = pd.read_sql(
            sql=get_individual_records_query(record_col, record_name),
            con=conn
        )
    

In [13]:
df

,player,map,mode,date,mostDeathsInMatch
0,gitan,mp_cliffhanger,dem,2023-10-01,56.0
1,esterad,mp_paintball_rm,dem,2023-10-10,56.0
2,dawny_keczup,mp_paintball_rm,dem,2023-10-10,56.0
3,worried_blade115,mp_sm_gas_station,ctf,2023-10-02,43.0
4,notevidius,mp_sm_gas_station,ctf,2023-10-02,42.0
5,boruchomir,mp_paintball_rm,dem,2023-10-10,42.0


In [26]:
import pandas as pd
import matplotlib.pyplot as plt


def save_dataframe_as_table(dataframe, file_path, **kwargs):
    """
    Save a Pandas DataFrame as an image representing a table.

    Parameters:
    - dataframe: pd.DataFrame
        The DataFrame to be saved as a table.
    - file_path: str
        The path to save the PNG file.
    - **kwargs:
        Additional keyword arguments to be passed to the table function.

    Returns:
    None
    """
    fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the figure size as needed

    # Create a table from the DataFrame
    table = ax.table(cellText=dataframe.values,
                     colLabels=dataframe.columns,
                     cellLoc='center',
                     loc='center',
                     **kwargs)

    # Remove the axis for a cleaner look
    ax.axis('off')

    # Save the figure as PNG
    fig.savefig(file_path, bbox_inches='tight', pad_inches=0.05)
    plt.close()

In [23]:
save_dataframe_as_table(df, 'rec1.png')